In [1]:
import pymongo
MongoClient = pymongo.MongoClient
client = MongoClient()
from geopy.geocoders import Nominatim
import pandas as pd


In [ ]:
def create_db(name):
    db_clean = client.name

In [33]:
filterquery ={"$and":
    [
     {"$or":
        [
            {"category_code": "software"},
            {"category_code": "web"},
            {"category_code": "mobile"},
            {"category_code": "games_video"}
        ]},
      {"$and":
         [
             {"founded_year": {"$gt":2000}},
             {"$or":
              [
                 {"deadpooled_year":{"$eq": None}},
                 {"deadpooled_year":{"$exists":False}}
              ]
             }
         ]},
              {"$and":
       [
          {"relationships.title":{'$regex':'Software Engineer'}},
          {"$or":
             [
                 {"$or":
                     [
                         {"relationships.title":{'$regex':'[Dd]esigner'}},
                         {"relationships.title":{'$regex':'[Cc]reative'}},
                         {"relationships.title":{'$regex':'U[XI]'}},
                         {"relationships.title":{'$regex':'[Cc]reative'}},
                     ]},
                 {"relationships.title":{'$regex':'[Dd]evelopers'}},
                 {"$or":
                     [
                         {"relationships.title":{'$regex':'[Ee]xecutive'}},
                         {"relationships.title":{'$regex':'[Mm]anager'}}
                     ]} 
            ]
          }
       ]
      }
    ]
}

# cursor = db.companies.find(filterquery)
# res=list(map(lambda r: [r['name'],r["offices"]],cursor))
# for i in res:
#     for j in i[1]:
#         if not j['latitude'] or not j['longitude']:
#             geolocator = Nominatim(user_agent="tableau-project")
#             location = geolocator.geocode(j['address1']+","+j['city'])
#             print(location.address)
#             print((location.latitude, location.longitude))
#         else:
#             print((j['latitude'],j['longitude']))
db.companies.count(filterquery)


# {"relationships.title":/Executive/}

/home/luis/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:56: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere


81

In [37]:
def filterData(filterquery):
    cursor = db.companies.find(filterquery)
    geoCompanies = list(map(lambda c: convertCompanies(c),cursor))
    return geoCompanies
    
def toGeoJSON(array):
    coor=['','']
    for i in array:
        geolocator = Nominatim(user_agent="tableau-project")
        
        if not i['latitude'] or not i['longitude']:
            if i['address1'] != '' and i['city'] != '' and i['address1'] != None and i['city'] != None:
                location = geolocator.geocode(i['address1'])
                if location != None:
                    coor=[location.longitude,location.latitude]
            elif i['address1'] != '' and not i['address1']:
                location = geolocator.geocode(i['address1'])
                coor=[location.longitude,location.latitude]
        else:
            coor=[i['longitude'],i['latitude']]
    if coor != ['','']:
        return {
            "type":"Point",
            "coordinates":coor
        }

def convertCompanies(company):
    return {
        "name": company["name"],
        "homepage_url": company["homepage_url"],
        "twitter_username": company["twitter_username"],
        "number_of_employees": company["number_of_employees"],
        "products": company["products"],
        "competitions": company["competitions"],
        "total_money_raised": company["total_money_raised"],
        "category_code":company["category_code"],
        "acquisition": company["acquisition"],
        "position": toGeoJSON(company["offices"])
    }

geoCompanies = filterData(filterquery)

In [ ]:
# db.companies.create_index([("loc", pymongo.GEOSPHERE)])

### I suposse that startup is a company that have less than 10 and a new company is a company that was founded after 2000 because the max founded year is 2013

In [29]:
# #Obtain longitude and latitude from adress that not appear in database
# from geopy.geocoders import Nominatim
# geolocator = Nominatim(user_agent="tableau-project")
# location = geolocator.geocode("257-A, Prashasan Nagar, Road 72")
# print(location.address)
# print((location.latitude, location.longitude))

In [38]:
df = pd.DataFrame(geoCompanies)
df.head()
df.to_json('clean_companies.json',orient="records", lines=True)

In [80]:
db_clean = client.clean_companies
cursor_clean1 = db_clean.clean_companies.find()

In [106]:
filterquery_2=

In [126]:
print(db_clean.clean_companies.count({"position":{"$eq":None}}))
def near_companies(cursor_cl):
    cursor_clean=[]
    for i in cursor_cl:
        if i['position'] != None:
            coord=i['position']['coordinates']
            cursor_clean.append(db_clean.clean_companies.count({"$and":
        [
            {"position":{"$ne":None}},
            {
              "position": {
                "$near": {
                  "$geometry": {
                    "type": "Point",
                    "coordinates": coord
                  },
                  "$minDistance": 0,
                  "$maxDistance":1000
                }
              }
            }
        ]
      }))
    return len(cursor_clean)
#             print(i['name'],coord,cursor_clean)
print(near_companies(client.clean_companies.clean_companies.find()))
# print(list(map(lambda r: r['name'],cursor_clean)))

2
79


/home/luis/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.
/home/luis/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere


In [ ]:
# def filterData(filterquery):
#     cursor = db.companies.find(filterquery)
#     geoCompanies = list(map(lambda c: convertCompanies(c),cursor))
#     return geoCompanies
    
# def toGeoJSON(array):
#     for i in array:
#         if not i['latitude'] or not i['longitude']:
#             geolocator = Nominatim(user_agent="tableau-project")
#             location = geolocator.geocode(i['address1']+","+i['city'])
#             coor=[location.longitude,location.latitude]
#         else:
#             coor=[i['longitude'],i['latitude']]
#     return {
#         "type":"Point",
#         "coordinates":coor
#     }

# def convertCompanies(company):
#     return {
#         "name": company["name"],
#         "category_code":company["category_code"],
#         "position": toGeoJSON(company["offices"])
#     }

# geoCompanies = filterData(filterquery)
# print(geoCompanies)